
# 🧪 05 — MCP Hands-On (Node.js / TypeScript Server, Option A)

This notebook gives you a **full Node.js / TypeScript MCP-style server skeleton**, mirroring the Python version in `04_MCP_HandsOn_Python_Server.ipynb`.

Goals:

- Provide a **clean, production-style project layout** for a Node MCP server
- Implement:
  - a **tool base class & registry**
  - **filesystem tools** (sandboxed)
  - **HTTP fetch tool** (safe, allow-listed)
  - a minimal **JSON-over-stdio loop** you can later replace with a real MCP transport
- Keep everything **framework-agnostic**, so you can plug it into:
  - a real MCP SDK
  - a custom transport
  - tests / CLIs



## 1. Target Project Layout (Node / TypeScript)

We’ll assume this structure under `src/node_mcp_template/`:

```bash
node_mcp_template/
  src/
    core/
      types.ts
      tool.ts
      registry.ts
    tools/
      filesystem.ts
      httpApi.ts
    server/
      logger.ts
      loop.ts
  package.json
  tsconfig.json
  README.md
```

This notebook will give you **code for all of these files**, which you can:

1. Copy from the cells into their respective files
2. Build with `tsc`
3. Run via `node dist/server/loop.js` (or similar)



## 2. Core Types (types.ts)

We define some basic TypeScript types to mirror the Python side:

- `ToolInput`
- `ToolSuccess`
- `ToolError`
- `ToolResult`

These will be reused across all tools and the registry.


In [ ]:
// src/core/types.ts

export interface ToolInput {
  payload: Record<string, unknown>;
}

export interface ToolSuccess {
  ok: true;
  data: unknown;
  meta?: Record<string, unknown>;
}

export interface ToolError {
  ok: false;
  error_code: string;
  message: string;
  details?: Record<string, unknown>;
}

export type ToolResult = ToolSuccess | ToolError;

export function success(
  data: unknown,
  meta: Record<string, unknown> = {}
): ToolSuccess {
  return { ok: true, data, meta };
}

export function error(
  error_code: string,
  message: string,
  details: Record<string, unknown> = {}
): ToolError {
  return { ok: false, error_code, message, details };
}



## 3. Tool Base Class & Registry (tool.ts, registry.ts)

We define a base `Tool` class and a simple `ToolRegistry`:

- Each tool:
  - has a `name`
  - returns a description (for listing tools)
  - validates inputs
  - implements `run()`

- The registry:
  - registers tools
  - retrieves tools by name
  - lists tool descriptions


In [ ]:
// src/core/tool.ts

import type { ToolInput, ToolResult, ToolError } from "./types.js";
import { error } from "./types.js";

export interface ToolDescription {
  name: string;
  description: string;
  input_schema: Record<string, unknown>;
}

export abstract class Tool {
  abstract readonly name: string;

  describe(): ToolDescription {
    return {
      name: this.name,
      description: "Base tool (override in subclass)",
      input_schema: {
        type: "object",
        properties: {},
      },
    };
  }

  validateInput(payload: Record<string, unknown>): ToolInput | ToolError {
    // Default implementation: no validation
    return { payload };
  }

  abstract run(input: ToolInput): Promise<ToolResult> | ToolResult;
}


In [ ]:
// src/core/registry.ts

import type { ToolDescription } from "./tool.js";
import type { Tool } from "./tool.js";

export class ToolRegistry {
  private tools = new Map<string, Tool>();

  register(tool: Tool): void {
    if (this.tools.has(tool.name)) {
      throw new Error(`Tool with name '${tool.name}' is already registered`);
    }
    this.tools.set(tool.name, tool);
  }

  get(name: string): Tool | undefined {
    return this.tools.get(name);
  }

  list(): ToolDescription[] {
    return Array.from(this.tools.values()).map((tool) => tool.describe());
  }
}

// You can create a global registry instance in server/loop.ts or a dedicated module.



## 4. Filesystem Tools (filesystem.ts)

We will implement:

1. `ListFilesTool` → `list_files`
2. `ReadFileTool` → `read_file`

They will:

- Operate under a sandboxed `BASE_DIR`
- Prevent path traversal outside base
- Offer basic pagination/truncation semantics


In [ ]:
// src/tools/filesystem.ts

import fs from "node:fs";
import path from "node:path";
import { fileURLToPath } from "node:url";

import type { ToolInput, ToolResult, ToolError } from "../core/types.js";
import { error, success } from "../core/types.js";
import { Tool } from "../core/tool.js";

const __filename = fileURLToPath(import.meta.url);
const __dirname = path.dirname(__filename);

// You can adjust this to any sandbox root
export const BASE_DIR = path.resolve(__dirname, "../../data_root");

function ensureBaseDir() {
  if (!fs.existsSync(BASE_DIR)) {
    fs.mkdirSync(BASE_DIR, { recursive: true });
  }
}

ensureBaseDir();

function resolveSafePath(relativePath: string): string {
  const candidate = path.resolve(BASE_DIR, relativePath);
  if (!candidate.startsWith(BASE_DIR)) {
    throw new Error("Path is outside allowed base directory");
  }
  return candidate;
}

export class ListFilesTool extends Tool {
  readonly name = "list_files";

  describe() {
    return {
      name: this.name,
      description:
        "List files under a relative directory path, sandboxed under BASE_DIR.",
      input_schema: {
        type: "object",
        properties: {
          path: { type: "string", description: "Relative directory path" },
          recursive: {
            type: "boolean",
            description: "Whether to recurse into subdirectories",
          },
          max_items: {
            type: "integer",
            description: "Maximum number of entries to return",
          },
        },
        required: ["path"],
      },
    };
  }

  validateInput(payload: Record<string, unknown>): ToolInput | ToolError {
    const pathVal = payload["path"];
    if (typeof pathVal !== "string") {
      return error("validation_error", "'path' must be a string", {
        field: "path",
      });
    }

    const recursive =
      typeof payload["recursive"] === "boolean"
        ? (payload["recursive"] as boolean)
        : false;

    let maxItems = 100;
    if (typeof payload["max_items"] === "number") {
      maxItems = payload["max_items"] | 0;
    }
    if (!Number.isInteger(maxItems) || maxItems <= 0) {
      return error("validation_error", "'max_items' must be a positive integer", {
        field: "max_items",
      });
    }

    return {
      payload: {
        path: pathVal,
        recursive,
        max_items: maxItems,
      },
    };
  }

  run(input: ToolInput): ToolResult {
    try {
      const { path: relPath, recursive, max_items } = input
        .payload as {
        path: string;
        recursive: boolean;
        max_items: number;
      };

      const base = resolveSafePath(relPath);
      if (!fs.existsSync(base)) {
        return error("not_found", `Path does not exist: ${relPath}`);
      }

      const results: Array<Record<string, unknown>> = [];

      const stat = fs.statSync(base);
      if (stat.isFile()) {
        results.push({
          path: path.relative(BASE_DIR, base),
          is_dir: false,
          size: stat.size,
        });
      } else {
        if (recursive) {
          const walk = (p: string) => {
            const entries = fs.readdirSync(p, { withFileTypes: true });
            for (const entry of entries) {
              const full = path.join(p, entry.name);
              const rel = path.relative(BASE_DIR, full);
              if (entry.isDirectory()) {
                results.push({
                  path: rel,
                  is_dir: true,
                  size: null,
                });
                if (results.length >= max_items) return;
                walk(full);
                if (results.length >= max_items) return;
              } else {
                const s = fs.statSync(full);
                results.push({
                  path: rel,
                  is_dir: false,
                  size: s.size,
                });
                if (results.length >= max_items) return;
              }
            }
          };
          walk(base);
        } else {
          const entries = fs.readdirSync(base, { withFileTypes: true });
          for (const entry of entries) {
            const full = path.join(base, entry.name);
            const rel = path.relative(BASE_DIR, full);
            if (entry.isDirectory()) {
              results.push({
                path: rel,
                is_dir: true,
                size: null,
              });
            } else {
              const s = fs.statSync(full);
              results.push({
                path: rel,
                is_dir: false,
                size: s.size,
              });
            }
            if (results.length >= max_items) break;
          }
        }
      }

      const meta = {
        base_dir: BASE_DIR,
        count: results.length,
        truncated: results.length >= max_items,
      };
      return success(results, meta);
    } catch (err: any) {
      return error("internal_error", String(err?.message || err), {
        stack: err?.stack,
      });
    }
  }
}

export class ReadFileTool extends Tool {
  readonly name = "read_file";

  describe() {
    return {
      name: this.name,
      description:
        "Read a file (text) up to max_bytes, sandboxed under BASE_DIR.",
      input_schema: {
        type: "object",
        properties: {
          path: { type: "string", description: "Relative file path" },
          max_bytes: { type: "integer", description: "Max bytes to read" },
        },
        required: ["path"],
      },
    };
  }

  validateInput(payload: Record<string, unknown>): ToolInput | ToolError {
    const pathVal = payload["path"];
    if (typeof pathVal !== "string") {
      return error("validation_error", "'path' must be a string", {
        field: "path",
      });
    }

    let maxBytes = 4096;
    if (typeof payload["max_bytes"] === "number") {
      maxBytes = payload["max_bytes"] | 0;
    }
    if (!Number.isInteger(maxBytes) || maxBytes <= 0) {
      return error("validation_error", "'max_bytes' must be a positive integer", {
        field: "max_bytes",
      });
    }

    return {
      payload: {
        path: pathVal,
        max_bytes: maxBytes,
      },
    };
  }

  run(input: ToolInput): ToolResult {
    try {
      const { path: relPath, max_bytes } = input.payload as {
        path: string;
        max_bytes: number;
      };

      const full = resolveSafePath(relPath);
      if (!fs.existsSync(full) || !fs.statSync(full).isFile()) {
        return error("not_found", `File not found: ${relPath}`);
      }

      const buffer = fs.readFileSync(full);
      const slice = buffer.subarray(0, max_bytes);
      let text: string;
      try {
        text = slice.toString("utf8");
      } catch {
        text = slice.toString("latin1");
      }

      const truncated = buffer.byteLength > max_bytes;

      const meta = {
        path: path.relative(BASE_DIR, full),
        bytes_read: slice.byteLength,
        total_size: buffer.byteLength,
        truncated,
      };

      return success({ text }, meta);
    } catch (err: any) {
      return error("internal_error", String(err?.message || err), {
        stack: err?.stack,
      });
    }
  }
}



## 5. HTTP API Tool (httpApi.ts)

Similar to the Python version, we’ll implement a simple `HttpFetchTool` that:

- Only allows `https://`
- Restricts hostnames to an allow-list
- Performs **GET** requests only
- Truncates body to `max_bytes`


In [ ]:
// src/tools/httpApi.ts

import { URL } from "node:url";
import https from "node:https";

import type { ToolInput, ToolResult, ToolError } from "../core/types.js";
import { error, success } from "../core/types.js";
import { Tool } from "../core/tool.js";

const ALLOWED_HTTP_HOSTS = new Set<string>([
  "api.github.com",
  "jsonplaceholder.typicode.com",
  // add more allowed hosts here
]);

function fetchHttps(urlStr: string, maxBytes: number): Promise<{
  status: number;
  headers: Record<string, string>;
  body: string;
  truncated: boolean;
}> {
  return new Promise((resolve, reject) => {
    const url = new URL(urlStr);
    const options: https.RequestOptions = {
      method: "GET",
      hostname: url.hostname,
      path: url.pathname + url.search,
      headers: {
        "User-Agent": "node-mcp-template",
        Accept: "application/json, text/plain;q=0.9, */*;q=0.8",
      },
      timeout: 10000,
    };

    const req = https.request(options, (res) => {
      if (!res.statusCode) {
        reject(new Error("No status code from upstream"));
        return;
      }
      const status = res.statusCode;
      const headers: Record<string, string> = {};
      for (const [k, v] of Object.entries(res.headers)) {
        if (typeof v === "string") {
          headers[k.toLowerCase()] = v;
        } else if (Array.isArray(v)) {
          headers[k.toLowerCase()] = v.join(", ");
        }
      }

      const chunks: Buffer[] = [];
      let total = 0;
      let truncated = false;

      res.on("data", (chunk: Buffer) => {
        if (truncated) {
          return;
        }
        total += chunk.length;
        if (total > maxBytes) {
          const needed = maxBytes - (total - chunk.length);
          if (needed > 0) {
            chunks.push(chunk.subarray(0, needed));
          }
          truncated = true;
        } else {
          chunks.push(chunk);
        }
      });

      res.on("end", () => {
        const buffer = Buffer.concat(chunks);
        let body: string;
        try {
          body = buffer.toString("utf8");
        } catch {
          body = buffer.toString("latin1");
        }
        resolve({ status, headers, body, truncated });
      });
    });

    req.on("error", (err) => reject(err));
    req.on("timeout", () => {
      req.destroy(new Error("Upstream timeout"));
    });

    req.end();
  });
}

export class HttpFetchTool extends Tool {
  readonly name = "http_fetch";

  describe() {
    return {
      name: this.name,
      description:
        "Fetch content from a limited set of HTTPS APIs (GET only, allow-listed hosts).",
      input_schema: {
        type: "object",
        properties: {
          url: { type: "string", description: "Full https URL" },
          max_bytes: {
            type: "integer",
            description: "Maximum bytes to read from body",
          },
        },
        required: ["url"],
      },
    };
  }

  validateInput(payload: Record<string, unknown>): ToolInput | ToolError {
    const urlVal = payload["url"];
    if (typeof urlVal !== "string") {
      return error("validation_error", "'url' must be a string", {
        field: "url",
      });
    }

    if (!urlVal.startsWith("https://")) {
      return error("validation_error", "Only https:// URLs are allowed", {
        field: "url",
      });
    }

    let maxBytes = 4096;
    if (typeof payload["max_bytes"] === "number") {
      maxBytes = payload["max_bytes"] | 0;
    }
    if (!Number.isInteger(maxBytes) || maxBytes <= 0) {
      return error("validation_error", "'max_bytes' must be a positive integer", {
        field: "max_bytes",
      });
    }

    let hostname: string;
    try {
      const u = new URL(urlVal);
      hostname = u.hostname;
    } catch {
      return error("validation_error", "Invalid URL", { field: "url" });
    }

    if (!ALLOWED_HTTP_HOSTS.has(hostname)) {
      return error(
        "forbidden_host",
        `Host '${hostname}' is not allowed in ALLOWED_HTTP_HOSTS`
      );
    }

    return {
      payload: {
        url: urlVal,
        max_bytes: maxBytes,
      },
    };
  }

  async run(input: ToolInput): Promise<ToolResult> {
    const { url, max_bytes } = input.payload as {
      url: string;
      max_bytes: number;
    };
    try {
      const { status, headers, body, truncated } = await fetchHttps(
        url,
        max_bytes
      );
      return success(
        {
          status,
          headers,
          body,
        },
        {
          url,
          truncated,
          max_bytes,
        }
      );
    } catch (err: any) {
      return error("http_error", String(err?.message || err), {
        stack: err?.stack,
      });
    }
  }
}



## 6. Logger Setup (logger.ts)

We’ll create a small logger utility that writes to `stderr` with timestamps.


In [ ]:
// src/server/logger.ts

/* Simple logger utility for the Node MCP template.
   You can replace this with pino / winston / another logging library later.
*/

export type LogLevel = "debug" | "info" | "warn" | "error";

function log(level: LogLevel, message: string, meta?: Record<string, unknown>) {
  const ts = new Date().toISOString();
  const base = { ts, level, message };
  const combined = meta ? { ...base, meta } : base;
  // Write as single-line JSON for easy parsing
  process.stderr.write(JSON.stringify(combined) + "\n");
}

export const logger = {
  debug: (msg: string, meta?: Record<string, unknown>) =>
    log("debug", msg, meta),
  info: (msg: string, meta?: Record<string, unknown>) =>
    log("info", msg, meta),
  warn: (msg: string, meta?: Record<string, unknown>) =>
    log("warn", msg, meta),
  error: (msg: string, meta?: Record<string, unknown>) =>
    log("error", msg, meta),
};



## 7. JSON-Over-STDIO Server Loop (loop.ts)

Like the Python version, we’ll implement a simple **JSON-RPC style loop**:

- Read lines from `stdin`
- Parse JSON
- Support:
  - `{"id": "...", "type": "list_tools"}`
  - `{"id": "...", "type": "call_tool", "tool": "list_files", "input": {...}}`
- Write JSON responses on `stdout`, one per line

Again, this is **not** a full MCP protocol implementation, but:

- It exercises:
  - tool registry
  - validation
  - execution
  - error handling
- You can later replace the transport with a real MCP SDK/server.


In [ ]:
// src/server/loop.ts

import readline from "node:readline";

import { ToolRegistry } from "../core/registry.js";
import type { ToolResult } from "../core/types.js";
import { error } from "../core/types.js";
import { logger } from "./logger.js";
import { ListFilesTool, ReadFileTool } from "../tools/filesystem.js";
import { HttpFetchTool } from "../tools/httpApi.js";

// Build registry and register tools
const registry = new ToolRegistry();
registry.register(new ListFilesTool());
registry.register(new ReadFileTool());
registry.register(new HttpFetchTool());

interface ListToolsRequest {
  id: string | number | null;
  type: "list_tools";
}

interface CallToolRequest {
  id: string | number | null;
  type: "call_tool";
  tool: string;
  input?: Record<string, unknown>;
}

type Request = ListToolsRequest | CallToolRequest;

function isListToolsRequest(req: any): req is ListToolsRequest {
  return req && req.type === "list_tools";
}

function isCallToolRequest(req: any): req is CallToolRequest {
  return req && req.type === "call_tool" && typeof req.tool === "string";
}

async function handleRequest(req: Request): Promise<Record<string, unknown>> {
  const id = req.id ?? null;

  if (isListToolsRequest(req)) {
    const tools = registry.list();
    return {
      id,
      ok: true,
      type: "list_tools_result",
      tools,
    };
  }

  if (isCallToolRequest(req)) {
    const toolName = req.tool;
    const inputPayload = req.input ?? {};
    const tool = registry.get(toolName);
    if (!tool) {
      return {
        id,
        ok: false,
        error_code: "tool_not_found",
        message: `Tool '${toolName}' not found`,
      };
    }

    const started = Date.now();
    logger.info(`Calling tool '${toolName}'`, { input: inputPayload });

    const validated = tool.validateInput(inputPayload);
    let result: ToolResult;
    if (!validated || (validated as ToolResult).ok === false) {
      // validation returned an error
      result = validated as ToolResult;
    } else {
      result = await tool.run(validated as any);
    }

    const durationMs = Date.now() - started;
    logger.info(`Tool '${toolName}' finished`, {
      duration_ms: durationMs,
      ok: result.ok,
    });

    if (result.ok) {
      return {
        id,
        ok: true,
        result,
        duration_ms: durationMs,
      };
    } else {
      return {
        id,
        ok: false,
        error: result,
        duration_ms: durationMs,
      };
    }
  }

  return {
    id,
    ok: false,
    error_code: "unknown_request_type",
    message: `Unsupported request type: ${(req as any).type}`,
  };
}

export function startServerLoop() {
  logger.info("Node MCP-style server loop starting. Waiting for JSON on stdin...");
  const rl = readline.createInterface({
    input: process.stdin,
    crlfDelay: Infinity,
  });

  rl.on("line", async (line) => {
    const trimmed = line.trim();
    if (!trimmed) {
      return;
    }

    let parsed: any;
    try {
      parsed = JSON.parse(trimmed);
    } catch (err: any) {
      logger.error("Failed to parse JSON from line", { line: trimmed });
      const resp = {
        id: null,
        ok: false,
        error_code: "invalid_json",
        message: "Could not parse request as JSON",
      };
      process.stdout.write(JSON.stringify(resp) + "\n");
      return;
    }

    let response: Record<string, unknown>;
    try {
      response = await handleRequest(parsed);
    } catch (err: any) {
      logger.error("Unhandled error in handleRequest", {
        error: String(err?.message || err),
      });
      response = {
        id: parsed?.id ?? null,
        ok: false,
        error_code: "internal_server_error",
        message: String(err?.message || err),
      };
    }

    process.stdout.write(JSON.stringify(response) + "\n");
  });

  rl.on("close", () => {
    logger.info("Input stream closed. Exiting server loop.");
    process.exit(0);
  });
}

// Allow `node dist/server/loop.js` to start the loop directly.
if (import.meta.url === `file://${process.argv[1]}`) {
  startServerLoop();
}



## 8. package.json and tsconfig.json (Template)

You can start with something like this:


In [ ]:
// package.json (example for node_mcp_template)

{
  "name": "node-mcp-template",
  "version": "0.1.0",
  "type": "module",
  "main": "dist/server/loop.js",
  "scripts": {
    "build": "tsc",
    "start": "node dist/server/loop.js"
  },
  "devDependencies": {
    "typescript": "^5.0.0"
  }
}


In [ ]:
// tsconfig.json (example for node_mcp_template)

{
  "compilerOptions": {
    "target": "ES2020",
    "module": "Node16",
    "moduleResolution": "Node16",
    "outDir": "dist",
    "rootDir": "src",
    "strict": true,
    "esModuleInterop": true,
    "forceConsistentCasingInFileNames": true,
    "skipLibCheck": true
  },
  "include": ["src"]
}



## 9. How to Use This Template

1. In your repo, create:

```bash
cd world-class-mcp-foundation
mkdir -p src/node_mcp_template/src/core
mkdir -p src/node_mcp_template/src/tools
mkdir -p src/node_mcp_template/src/server
```

2. Copy code from this notebook into:

- `src/node_mcp_template/src/core/types.ts`
- `src/node_mcp_template/src/core/tool.ts`
- `src/node_mcp_template/src/core/registry.ts`
- `src/node_mcp_template/src/tools/filesystem.ts`
- `src/node_mcp_template/src/tools/httpApi.ts`
- `src/node_mcp_template/src/server/logger.ts`
- `src/node_mcp_template/src/server/loop.ts`
- `src/node_mcp_template/package.json`
- `src/node_mcp_template/tsconfig.json`

3. Install dependencies and build:

```bash
cd src/node_mcp_template
npm install
npm run build
npm start
```

4. Send JSON requests on stdin (for testing), for example:

```bash
echo '{"id": 1, "type": "list_tools"}' | node dist/server/loop.js
```

or from another script/process.

---

Later, when you integrate with a **real MCP SDK**:

- You’ll reuse:
  - Tool base class
  - Tool implementations
  - Registry concept
  - Logger
- You’ll swap out:
  - the stdin/stdout loop
  - request/response wire format

The important part is that your **tool logic and safety patterns** are already in a clean, reusable form.
